In [1]:
import pandas as pd

In [5]:
df=pd.read_csv('./product_csv_files/Beauty and Grooming.csv')
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,WOW Skin Science Vitamin C Serum for Skin whit...,beauty & health,Beauty & Grooming,https://m.media-amazon.com/images/I/71-RGUlIko...,https://www.amazon.in/WOW-Skin-Science-Vitamin...,3.9,"16,175",₹412,₹699
1,Dot & key Lip Plumping Mask Vitamin C + E (Lin...,beauty & health,Beauty & Grooming,https://m.media-amazon.com/images/I/51kfbv8mGi...,https://www.amazon.in/Dot-Key-PLUMPING-SLEEPIN...,3.9,"2,251",₹185,₹295
2,Park Avenue Signature Collection Premium Body ...,beauty & health,Beauty & Grooming,https://m.media-amazon.com/images/I/81thnD6HwD...,https://www.amazon.in/Park-Avenue-Super-Saver-...,4.2,"10,299",₹422.40,₹480
3,Sanfe Spotlite Sensitive Body Serum For Dark U...,beauty & health,Beauty & Grooming,https://m.media-amazon.com/images/I/61-1+OUySD...,https://www.amazon.in/Sanfe-Sensitive-Niacinna...,4.0,421,₹378,₹399
4,"Temperia Leg, Heel & Foot Scrubber for Dead Sk...",beauty & health,Beauty & Grooming,https://m.media-amazon.com/images/I/71SSLoMZuZ...,https://www.amazon.in/Temperia-Metal-Scrubber-...,4.1,"1,806",₹274,₹599


In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [11]:
start_time = time.time()
timeout = 40  # Timeout in seconds

# Start the Selenium driver
driver = webdriver.Chrome()  # Make sure you have the correct WebDriver for your browser

for index, row in df.iterrows():
    amazon_url = row['link']
    print(f"Processing URL: {amazon_url}")
    driver.get(amazon_url)
    time.sleep(2)  # Allow time for JavaScript to load
    
    # Extract product title
    try:
        if time.time() - start_time < timeout:
            title_tag = driver.find_element(By.ID, 'productTitle')
            df.at[index, 'Product Title'] = title_tag.text.strip()
        else:
            df.at[index, 'Product Title'] = None
    except:
        df.at[index, 'Product Title'] = None

    # Extract price
    try:
        if time.time() - start_time < timeout:
            price_tag = driver.find_element(By.CSS_SELECTOR, '.a-price-whole')
            df.at[index, 'Price'] = price_tag.text.strip()
        else:
            df.at[index, 'Price'] = None
    except:
        df.at[index, 'Price'] = None

    # Extract description
    try:
        if time.time() - start_time < timeout:
            description_tags = driver.find_elements(By.CSS_SELECTOR, 'ul.a-unordered-list.a-vertical.a-spacing-mini li')
            descriptions = [tag.text.strip() for tag in description_tags]
            df.at[index, 'Description'] = ', '.join(descriptions)
        else:
            df.at[index, 'Description'] = None
    except:
        df.at[index, 'Description'] = None

    # Extract additional product details
    try:
        if time.time() - start_time < timeout:
            print("Trying to find details table...")
            # Wait for the product details table to be present
            details_table = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'productDetails_detailBullets_sections1'))
            )
            print("Details table found.")
            
            rows = details_table.find_elements(By.TAG_NAME, 'tr')
            for row1 in rows:
                if time.time() - start_time >= timeout:
                    print("Timeout reached while processing rows.")
                    break
                try:
                    header = row1.find_element(By.TAG_NAME, 'th').text.strip()
                    value = row1.find_element(By.TAG_NAME, 'td').text.strip()
                    print(f"Found detail: {header} - {value}")
                    if header in ['Manufacturer', 'Packer', 'Importer', 'Item Weight', 'Item Dimensions LxWxH', 'Net Quantity', 'Included Components', 'Generic Name', 'Best Sellers Rank']:
                        df.at[index, header] = value
                except Exception as e:
                    print(f"Error processing row: {e}")
                    continue
    except Exception as e:
        print(f"Error finding details table: {e}")
        pass

# Close the Selenium driver
driver.quit()

# Display or save the updated DataFrame
#print(df)
# df.to_csv('scraped_data.csv', index=False)  # Uncomment to save to CSV

Processing URL: https://www.amazon.in/WOW-Skin-Science-Vitamin-Serum/dp/B085T9LGFV/ref=sr_1_385?qid=1679214946&s=beauty&sr=1-385
Trying to find details table...
Error finding details table: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF659AF38A5+3004357]
	(No symbol) [0x00007FF659789970]
	(No symbol) [0x00007FF65963582A]
	(No symbol) [0x00007FF659685B8E]
	(No symbol) [0x00007FF659685E7C]
	(No symbol) [0x00007FF6596CEC27]
	(No symbol) [0x00007FF6596ABC1F]
	(No symbol) [0x00007FF6596CBA4C]
	(No symbol) [0x00007FF6596AB983]
	(No symbol) [0x00007FF659677628]
	(No symbol) [0x00007FF659678791]
	GetHandleVerifier [0x00007FF659B1A00D+3161901]
	GetHandleVerifier [0x00007FF659B6E060+3506048]
	GetHandleVerifier [0x00007FF659B6400D+3465005]
	GetHandleVerifier [0x00007FF6598E0EEB+830987]
	(No symbol) [0x00007FF65979467F]
	(No symbol) [0x00007FF6597909D4]
	(No symbol) [0x00007FF659790B6D]
	(No symbol) [0x00007FF659780149]
	BaseThreadInitThunk [0x00007FF9173B259D+29]
	RtlUserThreadStart [0x00007

KeyboardInterrupt: 